### Jasmine Dumas (1523905)
### CSC 478: Programming Machine Learning Applications - Autumn 2016
### [Assignment #4](http://facweb.cs.depaul.edu/mobasher/classes/csc478/Assignments/assign4.html)
### Due: Thursday, November 17, 2016

For this assignment you will experiment with Principal Component Analysis as a dimensionality reduction approach to assist in clustering high-dimensional data. You will also perform association rule mining using the implementation provided in the textbook (Machine Learning in Action, Chapter 11). Finally, you'll experiment with item-based recommendation for a joke recommender system.

## 1. PCA for Reduced Dimensionality in Clustering [Dataset: segmentation_data.zip]

For this problem you will use an image segmentation data set for clustering. You will experiment with using PCA as an approach to reduce dimensionality and noise in the data. You will compare the results of clustering the data with and without PCA using the provided image class assignments as the ground truth. The data set is divided into three files. The file "segmentation_data.txt" contains data about images with each line corresponding to one image. Each image is represented by 19 features (these are the columns in the data and correspond to the feature names in the file "segmentation_names.txt". The file "segmentation_classes.txt" contains the class labels (the type of image) and a numeric class label for each of the corresponding images in the data file. After clustering the image data, you will use the class labels to measure completeness and homogeneity of the generated clusters. The data set used in this problem is based on the Image Segmentation data set at the UCI Machine Learning Repository.

Your tasks in this problem are the following:

   1. Load in the image data matrix (with rows as images and columns as features). Also load in the numeric class labels from the segmentation class file. Using your favorite method (e.g., sklearn's min-max scaler), perform min-max normalization on the data matrix so that each feature is scaled to [0,1] range.
   2. Next, Perform Kmeans clustering (for this problem, use the Kmeans implementation in scikit-learn) on the image data (since there are a total 7 pre-assigned image classes, you should use K = 7 in your clustering). Use Euclidean distance as your distance measure for the clustering. Print the cluster centroids (use some formatting so that they are visually understandable). Compare your 7 clusters to the 7 pre-assigned classes by computing the Completeness and Homogeneity values of the generated clusters.
   3. Perform PCA on the normalized image data matrix. You may use the linear algebra package in Numpy or the Decomposition module in scikit-learn (the latter is much more efficient). Analyze the principal components to determine the number, r, of PCs needed to capture at least 95% of variance in the data. Then use these r components as features to transform the data into a reduced dimension space. [See the PCA Clustering Notebook from class for an example of how these steps are performed.]
   4. Perform Kmeans again, but this time on the lower dimensional transformed data. Then, compute the Completeness and Homogeneity values of the new clusters.
   5. Discuss your observations based on the comparison of the two clustering results.


In [119]:
import numpy as np
import pylab as pl
import pandas as pd
from sklearn.cluster import KMeans 
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import scale

In [57]:
#data_img = np.genfromtxt("segmentation_data/segmentation_data.txt",delimiter=",",dtype=float)
data_img = pd.read_csv("segmentation_data/segmentation_data.txt", header=None)
print(data_img.shape)

(2100, 19)


In [58]:
data_img.head(n=6)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,110.0,189.0,9,0.0,0.0,1.000000,0.666667,1.222222,1.186342,12.925926,10.888889,9.222222,18.666668,-6.111111,-11.111111,17.222221,18.666668,0.508139,1.910864
1,86.0,187.0,9,0.0,0.0,1.111111,0.720082,1.444444,0.750309,13.740741,11.666667,10.333334,19.222221,-6.222222,-10.222222,16.444445,19.222221,0.463329,1.941465
2,225.0,244.0,9,0.0,0.0,3.388889,2.195113,3.000000,1.520234,12.259259,10.333334,9.333334,17.111110,-5.777778,-8.777778,14.555555,17.111110,0.480149,1.987902
3,47.0,232.0,9,0.0,0.0,1.277778,1.254621,1.000000,0.894427,12.703704,11.000000,9.000000,18.111110,-5.111111,-11.111111,16.222221,18.111110,0.500966,1.875362
4,97.0,186.0,9,0.0,0.0,1.166667,0.691215,1.166667,1.005540,15.592592,13.888889,11.777778,21.111110,-5.111111,-11.444445,16.555555,21.111110,0.442661,1.863654
5,157.0,221.0,9,0.0,0.0,1.055556,0.646930,1.222222,0.620633,12.111111,10.222222,8.111112,18.000000,-5.666666,-12.000000,17.666666,18.000000,0.549180,1.877146


In [59]:
# not sure about how to read this file in. None arguement for dtype lets it figure out the type from each column
class_img = pd.read_csv("segmentation_data/segmentation_classes.txt", header=None, sep= "\t" )
print(class_img.shape)

(2100, 2)


In [60]:
class_img.head(n=6)

,0,1
0,GRASS,0
1,GRASS,0
2,GRASS,0
3,GRASS,0
4,GRASS,0
5,GRASS,0


#### What are the unique class labels?

In [70]:
np.unique(class_img[0])

array(['BRICKFACE', 'CEMENT', 'FOLIAGE', 'GRASS', 'PATH', 'SKY', 'WINDOW'], dtype=object)

In [61]:
# min-max normalization of data_img so that features on differnt scale's don't out power others
mins = data_img.min()
maxs = data_img.max()
print(mins)
print(maxs)

0     1.000000e+00
1     1.100000e+01
2     9.000000e+00
3     0.000000e+00
4     0.000000e+00
5     0.000000e+00
6     0.000000e+00
7     0.000000e+00
8    -1.589457e-08
9     0.000000e+00
10    0.000000e+00
11    0.000000e+00
12    0.000000e+00
13   -4.966667e+01
14   -1.244444e+01
15   -3.388889e+01
16    0.000000e+00
17    0.000000e+00
18   -3.044175e+00
dtype: float64
0      254.000000
1      251.000000
2        9.000000
3        0.333333
4        0.222222
5       29.222221
6      991.718400
7       44.722225
8     1386.329200
9      143.444440
10     137.111110
11     150.888890
12     142.555560
13       9.888889
14      82.000000
15      24.666666
16     150.888890
17       1.000000
18       2.912480
dtype: float64


In [62]:
data_img = (data_img - mins) / (maxs-mins)
print(data_img.head(n=5))

         0         1   2    3    4         5         6         7         8   \
0  0.430830  0.741667 NaN  0.0  0.0  0.034221  0.000672  0.027329  0.000856   
1  0.335968  0.733333 NaN  0.0  0.0  0.038023  0.000726  0.032298  0.000541   
2  0.885375  0.970833 NaN  0.0  0.0  0.115970  0.002213  0.067081  0.001097   
3  0.181818  0.920833 NaN  0.0  0.0  0.043726  0.001265  0.022360  0.000645   
4  0.379447  0.729167 NaN  0.0  0.0  0.039924  0.000697  0.026087  0.000725   

         9         10        11        12        13        14        15  \
0  0.090111  0.079417  0.061119  0.130943  0.731343  0.014118  0.872865   
1  0.095791  0.085089  0.068483  0.134840  0.729478  0.023529  0.859583   
2  0.085463  0.075365  0.061856  0.120031  0.736940  0.038824  0.827324   
3  0.088562  0.080227  0.059647  0.127046  0.748134  0.014118  0.855787   
4  0.108701  0.101297  0.078056  0.148090  0.748134  0.010588  0.861480   

         16        17        18  
0  0.123711  0.508139  0.831849  
1  0.1

In [64]:
# remove column 2 to avoid errors
data_img.drop(2, axis=1, inplace=True)

In [65]:
# kmeans clustering - intialize the object
kmeans = KMeans(n_clusters=7, max_iter=500, verbose=1) # initialization

In [66]:
# run the algorithm from sklearn module
kmeans.fit(data_img)

Initialization complete
Iteration  0, inertia 482.547
Iteration  1, inertia 374.046
Iteration  2, inertia 368.185
Iteration  3, inertia 366.310
Iteration  4, inertia 365.867
Iteration  5, inertia 365.683
Iteration  6, inertia 365.600
Iteration  7, inertia 365.539
Iteration  8, inertia 365.517
Iteration  9, inertia 365.504
Converged at iteration 9
Initialization complete
Iteration  0, inertia 512.278
Iteration  1, inertia 396.573
Iteration  2, inertia 393.851
Iteration  3, inertia 392.986
Iteration  4, inertia 392.545
Iteration  5, inertia 392.376
Iteration  6, inertia 391.900
Iteration  7, inertia 391.554
Iteration  8, inertia 391.369
Iteration  9, inertia 391.262
Converged at iteration 9
Initialization complete
Iteration  0, inertia 515.259
Iteration  1, inertia 392.028
Iteration  2, inertia 372.602
Iteration  3, inertia 360.207
Iteration  4, inertia 355.314
Iteration  5, inertia 353.872
Iteration  6, inertia 353.047
Iteration  7, inertia 351.980
Iteration  8, inertia 351.381
Iteratio

KMeans(copy_x=True, init='k-means++', max_iter=500, n_clusters=7, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=1)

In [71]:
# completeness and homogenity
c = kmeans.predict(data_img)

#### Let's also convert the target labels to integers:

In [94]:
t = np.zeros(len(class_img),dtype=int)
target = np.array(class_img[0])
print(t)
print(target)

[0 0 0 ..., 0 0 0]
['GRASS' 'GRASS' 'GRASS' ..., 'CEMENT' 'CEMENT' 'CEMENT']


In [95]:
# 'BRICKFACE', 'CEMENT', 'FOLIAGE', 'GRASS', 'PATH', 'SKY', 'WINDOW'
t[target == 'BRICKFACE'] = 0
t[target == 'CEMENT'] = 1
t[target == 'FOLIAGE'] = 2
t[target == 'GRASS'] = 3
t[target == 'PATH'] = 4
t[target == 'SKY'] = 5
t[target == 'WINDOW'] = 6

In [97]:
print(t)

[3 3 3 ..., 1 1 1]


Since we know what the classes are, we can evaluate clustering performance by using metrics that compare clusters to the actual classes. 
* **Homogeneity**: each cluster contains only members of a single class. 
* **Completeness**: all members of a given class are assigned to the same cluster.

The completeness score approaches 1 when most of the data points that are members of a given class are elements of the same cluster while the homogeneity score approaches 1 when all the clusters contain almost only data points that are member of a single class.

In [98]:
from sklearn.metrics import completeness_score, homogeneity_score

In [99]:
print(completeness_score(t,c))

0.613207471714


In [100]:
print(homogeneity_score(t,c))

0.611513043958


#### Cluster Centroids:

In [101]:
print(kmeans.cluster_centers_)

[[  2.56103232e-01   3.93468137e-01   7.45098018e-02   1.91176471e-02
    7.73428799e-02   4.10042113e-03   6.05736165e-02   4.96748748e-03
    1.48187304e-01   1.38085139e-01   1.85159837e-01   1.18538808e-01
    7.16906282e-01   3.44013845e-01   3.55458216e-01   1.85413237e-01
    4.11986237e-01   2.01889852e-01]
 [  5.35098814e-01   1.50166667e-01   2.77777769e-02   1.66666667e-03
    3.02281387e-02   5.42887957e-04   2.67660451e-02   5.86661900e-04
    8.23246433e-01   7.79716377e-01   8.94170356e-01   7.88760696e-01
    2.70665440e-01   6.66372551e-01   2.89386481e-01   8.94170356e-01
    2.11804171e-01   1.25065773e-01]
 [  5.13993692e-01   8.08936588e-01   7.74410751e-02   5.05050505e-03
    5.44737633e-02   1.40719343e-03   4.63349822e-02   1.40097198e-03
    1.08789943e-01   9.14029557e-02   9.24140773e-02   1.42676436e-01
    6.79161019e-01   7.90017879e-02   8.21286885e-01   1.34900800e-01
    4.14491323e-01   8.92332630e-01]
 [  7.50696260e-01   5.34564103e-01   3.99999988e

#### Principal Component Analysis

* Our goal is to generate a subset of componets that best represent the most variance within the data - there by reducing the dimensionality (predictors) and reducing complexity

In [103]:
from sklearn import decomposition

In [104]:
pca = decomposition.PCA(n_components=5)
DTtrans = pca.fit(data_img).transform(data_img)

In [105]:
np.set_printoptions(precision=2,suppress=True)
print(DTtrans)

[[-0.69  0.53  0.25  0.2  -0.08]
 [-0.67  0.51  0.34  0.17 -0.04]
 [-0.71  0.77 -0.16  0.01 -0.17]
 ..., 
 [-0.51 -0.13 -0.08 -0.1  -0.03]
 [-0.48 -0.09 -0.16 -0.11  0.  ]
 [-0.44 -0.11 -0.05 -0.15  0.02]]


In [108]:
print(pca.explained_variance_ratio_)

[ 0.61  0.13  0.1   0.05  0.04]


In [116]:
cp_5 = round(sum(pca.explained_variance_ratio_  * 100), 3) 
'It can be obsereved that the first 5 components capture (explain) {}% of the variance in the data.'.format(cp_5)

'It can be obsereved that the first 5 components capture (explain) 92.126% of the variance in the data.'

# Come back to later: Now, we can redo the clustering, but this time in the lower dimensional space:

In [130]:
#centroids_pca, clusters_pca = kMeans.kMeans(DTtrans, 3, kMeans.distCosine, kMeans.randCent)

In [128]:
kmeans = KMeans(n_clusters=7, max_iter=500, verbose=1)

In [129]:
kmeans.fit(DTtrans)

Initialization complete
Iteration  0, inertia 398.452
Iteration  1, inertia 238.390
Iteration  2, inertia 226.585
Iteration  3, inertia 224.826
Iteration  4, inertia 224.413
Iteration  5, inertia 224.283
Iteration  6, inertia 224.246
Converged at iteration 6
Initialization complete
Iteration  0, inertia 369.503
Iteration  1, inertia 246.351
Iteration  2, inertia 229.627
Iteration  3, inertia 225.675
Iteration  4, inertia 224.664
Iteration  5, inertia 224.387
Iteration  6, inertia 224.273
Iteration  7, inertia 224.238
Iteration  8, inertia 224.231
Converged at iteration 8
Initialization complete
Iteration  0, inertia 314.525
Iteration  1, inertia 231.409
Iteration  2, inertia 227.674
Iteration  3, inertia 226.066
Iteration  4, inertia 225.389
Iteration  5, inertia 225.206
Iteration  6, inertia 225.113
Iteration  7, inertia 225.092
Iteration  8, inertia 225.085
Iteration  9, inertia 225.077
Iteration 10, inertia 225.069
Converged at iteration 10
Initialization complete
Iteration  0, iner

KMeans(copy_x=True, init='k-means++', max_iter=500, n_clusters=7, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=1)

## 2. Association Rule Discovery [Dataset: playlists.zip]

For this problem you will experiment with association rule mining using Apriori algorithm discussed in class. You will use a modified version of the Apriori implementation in Machine Learning in Action (it has been modified to compute lift values for rules in addition to confidence). [See Associations Notebook from class for an example of using this module.]

The data set you will use is based on a music playlist data set obtained from Yes.com. [See the full description of this data]. We will only use a portion of this data. The provided data archive contains two files. The file "playlists.txt" contains on each line a sequence of songs played as part of one playlist. The songs are represented by integer values. The file "song_names.txt" contains the mapping between the integer codes and song tiles and artists (format of the song names is [song title]::[artist]). You will need both of these files to generate association rules.

Your tasks in this problem are the following:

1. Load the playlist data into a Python nested list, and the song_names data into an appropriate data structure.
2. Run Apriori on the playlist data using a min-support value of 0.002.
3. Generate rules from the frequent itemsets discovered in the previous step (you can try different metrics ('lift' or 'confidence' to see which gives you more useful and interesting results). At minimum, you should generate rules with a minimum lift value of 20.0. If you use confidence, you may want to set the min-confidence threshold to about 0.5.
4. Identify 3-4 rules and explain why you think they are interesting.

In [8]:
import helper
import apriori2
import apriori2 as ap

In [10]:
playLists = helper.loadPlayList('playlists/playlists.txt')

In [21]:
playLists

[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [9, 10, 11],
 [12, 13, 14, 15],
 [16, 17, 18],
 [19],
 [2],
 [20, 21, 22, 15, 23, 8, 24, 25],
 [26],
 [27, 18],
 [28, 29],
 [30, 3, 31],
 [20, 21],
 [2, 22, 32, 33, 34, 27, 24, 35],
 [26],
 [18, 36, 28, 15, 20],
 [37, 28, 15],
 [102],
 [172],
 [173, 53],
 [174],
 [175],
 [176],
 [199],
 [200],
 [201],
 [212],
 [215],
 [314, 211, 344],
 [230, 401],
 [257, 323, 286, 224, 461, 416, 274],
 [242],
 [222, 334, 288, 230],
 [295, 262, 219, 229, 215],
 [209, 207, 242, 434, 462, 220],
 [405],
 [311],
 [224, 400, 212, 243, 218, 359],
 [209, 213, 239, 264, 300, 237, 415],
 [367],
 [222, 292, 463, 262, 218, 357],
 [212, 351, 293],
 [238, 395, 209, 455, 232, 280, 254, 397, 257, 327],
 [249,
  464,
  209,
  277,
  458,
  353,
  262,
  341,
  246,
  283,
  211,
  360,
  355,
  285,
  241,
  465],
 [222, 352, 376],
 [237, 311, 348, 250, 212],
 [290],
 [206],
 [245, 300, 304, 230, 411, 445, 320, 242, 210, 331, 265, 204, 466, 257],
 [223, 306, 274, 253],
 [467, 372, 212, 27

In [22]:
songs = helper.createSongDictionary('playlists/song_names.txt', 2)

In [14]:
songs # some pretty dope songs :)

{0: b'Gucci-Time::Gucci-Mane',
 1: b'Aston-Martin-Music::Rick-Ross',
 2: b'Get-Back-Up::T.I.',
 3: b'Hot-Toddy::Usher',
 4: b'Whip-My-Hair::Willow',
 5: b'Down-On-Me::Jeremih',
 6: b'Black-And-Yellow::Wiz-Khalifa',
 7: b'Blowing-Me-Kisses::Soulja-Boy',
 8: b'Lay-It-Down::Lloyd',
 9: b"Who's-That-Chick::Rihanna",
 10: b'Like-A-G6::Far-East-Movement',
 11: b"DJ-Got-Us-Fallin'-In-Love::Usher",
 12: b'All-I-Wants-Is-You::Miguel',
 13: b'Champagne-Life::Ne-Yo',
 14: b'Find-Your-Love::Drake',
 15: b'Your-Love::Nicki-Minaj',
 16: b'Ms.-Jackson::Outkast',
 17: b'Close-To-You::Maxi-Priest',
 18: b'Right-Thru-Me::Nicki-Minaj',
 19: b'You-Make-Me-Wanna...::Usher',
 20: b"What's-My-Name::Rihanna",
 21: b'One-In-A-Million::Ne-Yo',
 22: b'Check-It-Out::Will.I.Am',
 23: b'Be-Without-You::Mary-J.-Blige',
 24: b'Bottoms-Up::Trey-Songz',
 25: b'There-Goes-My-Baby::Usher',
 26: b'Up-All-Night::Drake',
 27: b'Liquor-Store-Blues::Bruno-Mars',
 28: b'"Monster"::Kanye-West',
 29: b'Just-The-Way-You-Are::Brun

In [33]:
L, support = ap.apriori(playLists, 0.002)

In [34]:
ruleList1 = ap.generateRules(L, support, metric='confidence', minMetric=0.5)

In [35]:
ruleList1 = ap.generateRules(L, support, metric='confidence', minMetric=0.5)

In [37]:
ap.pntRules(ruleList1, songs)

In [38]:
ruleList2 = ap.generateRules(L, support, metric='lift', minMetric=20.0)

In [39]:
ap.pntRules(ruleList2, songs)

## 3. Item-Based Joke Recommendation [Dataset: jokes.zip]

For this problem you will use a modified version of the item-based recommender algorithm from Ch. 14 of Machine Learning in Action and use it on joke ratings data based on Jester Online Joke Recommender System. The modified version of the code is provided in the module itemBasedRec.py. Most of the module will be used as is, but you will add some additional functionality.

The data set contains two files. The file "modified_jester_data.csv" contains the ratings on 100 jokes by 1000 users (each row is a user profile). The ratings have been normalized to be between 1 and 21 (a 20-point scale), with 1 being the lowest rating. A zero indicated a missing rating. The file "jokes.csv" contains the joke ids mapped to the actual text of the jokes.

Your tasks in this problem are the following (please also see comments for the function stubs in the provided module):

1. Load in the joke ratings data and the joke text data into appropriate data structures.
2. Complete the definition for the function "test". This function iterates over all users and for each performs cross-validation on items (by calling the provided "cross_validate_user" function), and returns the error information necessary to compute Mean Absolute Error (MAE). Use this function to perform 5-fold cross-validation (i.e., 20% test-ratio) comparing MAE results using standard item-based collaborative filtering (based on the rating prediction function "standEst") with results using the SVD-based version of the rating item-based CF (using "svdEst" as the prediction engine). [Note: See comments provided in the module for hints on accomplishing these tasks.]
3. Write a new function "print_most_similar_jokes" which takes the joke ratings data, a query joke id, a parameter k for the number of nearest neighbors, and a similarity metric function, and prints the text of the query joke as well as the texts of the top k most similar jokes based on user ratings. [Note: For hints on how to accomplish this task, please see comments at the end of the provided module as well as comments for the provided stub function.]


In [25]:
import numpy as np
import itemBasedRec
from numpy import array

In [26]:
ratings = np.genfromtxt('jokes/modified_jester_data.csv',delimiter=',')

In [27]:
ratings.shape

(1000, 100)

In [28]:
jokes = np.genfromtxt('jokes/jokes.csv',delimiter=',',dtype=str)
jokes = np.array(jokes[:,1])

In [29]:
jokes[0]

'A man visits the doctor. The doctor says "I have bad news for you.You have cancer and Alzheimer\'s disease". The man replies "Well thank God I don\'t have cancer!"'

In [30]:
itemBasedRec.test(ratings,.2, itemBasedRec.svdEst)

/Users/jasminedumas/Desktop/depaul/CSC478/jupyterhw_4/itemBasedRec.py:70: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  test_indices = np.random.randint(0, len(rated_items_by_user), test_size)


Mean Absoloute Error for  <function svdEst at 0x103fb0510>  :  3.68653755981


3.6865375598075651

In [31]:
itemBasedRec.test(ratings, .2, itemBasedRec.standEst)

/Users/jasminedumas/Desktop/depaul/CSC478/jupyterhw_4/itemBasedRec.py:70: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  test_indices = np.random.randint(0, len(rated_items_by_user), test_size)


Mean Absoloute Error for  <function standEst at 0x103fb0400>  :  3.76615624372


3.7661562437205816

In [32]:
itemBasedRec.print_most_similar_jokes(ratings, jokes, 3, 5, itemBasedRec.pearsSim)

Selcted joke: 

Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.

The top 5 recommended jokes are: 

A dog walks into Western Union and asks the clerk to send a telegram. He fills out a form on which he writes down the telegram he wishes to send: "Bow wow wow Bow wow wow."The clerk says "You can add another 'Bow wow' for the same price."The dog responded "Now wouldn't that sound a little silly?"

Q:  What did the blind person say when given some matzah?A:  Who the hell wrote this?

Q. Did you hear about the dyslexic devil worshipper? A. He sold his soul to Santa.

Q. What is orange and sounds like a parrot?  A. A carrot.

A guy goes into confession and says to the priest "Father I'm 80 years old widower with 11 grandchildren. Last night I met two beautiful flight attendants. They took me home and I made love to both of them. Twice."The priest said: "Well my son when was the last time you were in confession?" "Never Father I'm 

---
fin.